In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import pmdarima as pm
from pmdarima.arima import auto_arima
print("pmdarima:", pm.__version__)
print("numpy:", np.__version__)

pmdarima: 2.0.4
numpy: 2.2.4


In [2]:
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [4]:
required_columns = {'Date', 'store', 'product', 'number_sold'}
if not required_columns.issubset(train_df.columns) or not required_columns.issubset(test_df.columns):
    raise ValueError(f"Ожидаются колонки: {required_columns}")

train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df['Date'] = pd.to_datetime(test_df['Date'])

stores = sorted(train_df['store'].unique())  # 7 магазинов
products = sorted(train_df['product'].unique())  # 10 товаров

In [ ]:
all_results = []

# --- Шаг 1: Цикл по (store, product) ---
for st in stores:
    for pr in products:
        # 1.1 Выделяем строки train_df / test_df для конкретного магазина и товара
        subset_train = train_df[(train_df['store'] == st) & (train_df['product'] == pr)].copy()
        subset_test = test_df[(test_df['store'] == st) & (test_df['product'] == pr)].copy()

        # Если вдруг subset_train пустой (маловероятно, но на всякий случай)
        if subset_train.empty:
            continue

        # 1.2 Устанавливаем индекс по дате, делаем временной ряд
        subset_train.set_index('Date', inplace=True)
        # Данные уже по дням без пропусков? Если пропусков дат нет, можно asfreq('D') не делать.
        # Но если хотим гарантированно выстроить весь календарь, можно раскомментировать:
        # subset_train = subset_train.asfreq('D', fill_value=0)

        # Аналогично для теста
        if not subset_test.empty:
            subset_test.set_index('Date', inplace=True)
            # subset_test = subset_test.asfreq('D', fill_value=0)  # если нужно выравнивать даты

        # --- Шаг 2: Подбираем (p,d,q) и (P,D,Q,m) через auto_arima ---
        # Предположим недельную сезонность m=7, можно менять по ситуации
        try:
            auto_model = auto_arima(
                subset_train['number_sold'],
                seasonal=True, m=7,
                d=None, D=None,
                start_p=0, max_p=3,
                start_q=0, max_q=3,
                start_P=0, max_P=2,
                start_Q=0, max_Q=2,
                information_criterion='aic',
                trace=False,
                error_action='ignore',
                suppress_warnings=True,
                stepwise=True
            )
            order = auto_model.order
            seasonal_order = auto_model.seasonal_order
        except Exception as e:
            print(f"auto_arima не сработал для store={st}, product={pr}. Ошибка: {e}")
            # Берём дефолтные параметры
            order = (1, 1, 1)
            seasonal_order = (0, 0, 0, 0)

        # --- Шаг 3: Обучаем SARIMAX ---
        try:
            model = SARIMAX(
                subset_train['number_sold'],
                order=order,
                seasonal_order=seasonal_order,
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            model_fit = model.fit(disp=False)
        except Exception as e:
            print(f"Не удалось обучить SARIMAX для store={st}, product={pr}. Ошибка: {e}")
            continue

        # --- Шаг 4: Прогноз на тестовом периоде и метрики ---
        if not subset_test.empty:
            forecast_steps = len(subset_test)
            forecast_object = model_fit.get_forecast(steps=forecast_steps)
            forecast_values = forecast_object.predicted_mean
            # Привяжем индекс прогноза к датам из subset_test
            forecast_values.index = subset_test.index

            # Рассчитываем метрики
            actual_values = subset_test['number_sold']

            # RMSE (если scikit-learn < 0.22, делаем вручную)
            mse = mean_squared_error(actual_values, forecast_values)
            rmse = np.sqrt(mse)

            # MAPE
            # (если есть нули в actual_values, придётся обрабатывать отдельно)
            try:
                mape_val = mean_absolute_percentage_error(actual_values, forecast_values)
            except ZeroDivisionError:
                mape_val = None

            # R2
            r2_val = r2_score(actual_values, forecast_values)

            # Сохраняем результат (можно записать всё одним DataFrame)
            temp_df = pd.DataFrame({
                'Date': forecast_values.index,
                'store': st,
                'product': pr,
                'Actual': actual_values,
                'Forecast': forecast_values,
                'RMSE': rmse,
                'MAPE': mape_val,
                'R2': r2_val,
            })
            all_results.append(temp_df)

C:\Users\admin\.conda\envs\arima_env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\admin\.conda\envs\arima_env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\admin\.conda\envs\arima_env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\admin\.conda\envs\arima_env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\admin\.conda\envs\arima_env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in

In [ ]:
if all_results:
    results_df = pd.concat(all_results, ignore_index=True)
    print("Пример итогового DataFrame:")
    print(results_df.head())


    results_df.to_csv("forecast_per_store_product.csv", index=False)
    print("\nРезультаты прогнозов сохранены в 'forecast_per_store_product.csv'.")

else:
    print("Нет результатов")